In [1]:
import pandas as pd
from plotting import CandlePlot
pd.set_option('display.max_columns', None)

In [2]:
df_ma_res = pd.read_csv("../data/ma_res_2023-12-04.csv")
df_ma_trades = pd.read_csv("../data/ma_trades_2023-12-04.csv")

In [3]:
df_ma_trades.head()

,Unnamed: 0,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MA_40,MA_200,MA_10,MA_80,MA_50,MA_20,MA_150,MA_120,MA_30,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,granularity,pair,GAIN_C,ma_l,ma_s,cross
0,221,2023-10-10 08:15:00+00:00,2117,1.05822,1.05888,1.05755,1.05886,1.05814,1.05880,1.05747,1.05878,1.05829,1.05895,1.05763,1.05894,1.056945,1.055727,1.056685,1.056204,1.056908,1.056553,1.055764,1.055497,1.056795,0.000131,-0.000047,1,-0.00095,-9.5,M15,EUR_USD,-9.5,MA_20,MA_10,MA_10_MA_20
1,237,2023-10-10 12:15:00+00:00,1814,1.05830,1.05837,1.05743,1.05791,1.05822,1.05830,1.05735,1.05784,1.05838,1.05844,1.05751,1.05798,1.057696,1.056086,1.058675,1.057192,1.057645,1.058982,1.055995,1.056020,1.058016,-0.000307,0.000020,-1,0.00241,-24.1,M15,EUR_USD,-33.6,MA_20,MA_10,MA_10_MA_20
2,245,2023-10-10 14:15:00+00:00,1883,1.06044,1.06068,1.05968,1.06032,1.06037,1.06060,1.05960,1.06024,1.06052,1.06075,1.05975,1.06040,1.058218,1.056234,1.059205,1.057631,1.057971,1.059157,1.056146,1.056447,1.058920,0.000047,-0.000265,1,-0.00045,-4.5,M15,EUR_USD,-38.1,MA_20,MA_10,MA_10_MA_20
3,262,2023-10-10 18:30:00+00:00,945,1.05967,1.05989,1.05938,1.05987,1.05959,1.05982,1.05930,1.05980,1.05975,1.05996,1.05946,1.05994,1.059793,1.056734,1.060333,1.058404,1.059249,1.060361,1.056676,1.057462,1.059746,-0.000028,0.000197,-1,0.00077,-7.7,M15,EUR_USD,-45.8,MA_20,MA_10,MA_10_MA_20
4,273,2023-10-10 21:15:00+00:00,43,1.06059,1.06072,1.06058,1.06064,1.06044,1.06062,1.06042,1.06055,1.06074,1.06084,1.06073,1.06074,1.059951,1.056806,1.060372,1.058798,1.059898,1.060277,1.057130,1.057984,1.060380,0.000095,-0.000018,1,0.00005,0.5,M15,EUR_USD,-45.3,MA_20,MA_10,MA_10_MA_20


In [4]:
df_ma_trades.granularity.unique()

array(['M15', 'M30', 'H1', 'H4'], dtype=object)

In [5]:
def filter_data_by_granularity(df_ma_res, df_ma_trades, granularity):
    """
    Filter DataFrame based on granularity and create copies.

    Parameters:
    - df_ma_res (pd.DataFrame): DataFrame for macro results.
    - df_ma_trades (pd.DataFrame): DataFrame for trade results.
    - granularity (str): Granularity to filter on.

    Returns:
    pd.DataFrame, pd.DataFrame: Filtered DataFrames for macro results and trade results.

    Description:
    This function takes two DataFrames, df_ma_res and df_ma_trades, and filters them based on the provided granularity.
    It creates copies of the filtered DataFrames and returns them as df_res_filtered and df_trades_filtered, respectively.
    """
    df_res_filtered = df_ma_res[df_ma_res.granularity == granularity].copy()
    df_trades_filtered = df_ma_trades[df_ma_trades.granularity == granularity].copy()
    return df_res_filtered, df_trades_filtered

df_ma_res_m15, df_ma_trades_m15 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'M15')
df_ma_res_m30, df_ma_trades_m30 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'M30')
df_ma_res_h1, df_ma_trades_h1 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'H1')
df_ma_res_h4, df_ma_trades_h4 = filter_data_by_granularity(df_ma_res, df_ma_trades, 'H4')


In [6]:
df_ma_res_m15.columns
df_ma_res_m30.columns
df_ma_res_h1.columns
df_ma_res_h4.columns

Index(['pair', 'num_trades', 'total_gain', 'mean_gain', 'min_gain', 'max_gain',
       'ma_l', 'ma_s', 'cross', 'granularity'],
      dtype='object')

In [7]:
def generate_cross_summary(df_ma_res, granularity):
    """
    Generate cross summary DataFrame based on granularity.

    Parameters:
    - df_ma_res (pd.DataFrame): DataFrame for macro results.
    - granularity (str): Granularity to filter on.

    Returns:
    pd.DataFrame: Cross summary DataFrame.

    Description:
    This function generates a cross summary DataFrame from the provided macro results DataFrame (df_ma_res)
    by filtering it based on the specified granularity. It creates a copy of the filtered DataFrame and extracts
    columns 'pair', 'number_of_trades', 'total_gain', and 'cross' to form the cross summary DataFrame.
    """
    df_res_filtered = df_ma_res[df_ma_res.granularity == granularity].copy()
    df_cross_summary = df_res_filtered[['pair', 'num_trades', 'total_gain', 'cross']].copy()
    return df_cross_summary

df_cross_summary_m15 = generate_cross_summary(df_ma_res, 'M15')
df_cross_summary_m30 = generate_cross_summary(df_ma_res, 'M30')
df_cross_summary_h1 = generate_cross_summary(df_ma_res, 'H1')
df_cross_summary_h4 = generate_cross_summary(df_ma_res, 'H4')


In [8]:
df_cross_summary_h4

,pair,num_trades,total_gain,cross
432,EUR_USD,212,-1465,MA_10_MA_20
433,EUR_USD,121,-1582,MA_10_MA_40
434,EUR_USD,113,-622,MA_20_MA_40
435,EUR_USD,129,-1206,MA_30_MA_40
436,EUR_USD,76,190,MA_10_MA_80
...,...,...,...,...
571,GBP_JPY,51,-5071,MA_10_MA_200
572,GBP_JPY,45,-5647,MA_20_MA_200
573,GBP_JPY,41,-6392,MA_30_MA_200
574,GBP_JPY,33,-3451,MA_40_MA_200


In [9]:
def group_and_sort(df):
    grouped_df = df.groupby(by="cross").sum()
    sorted_df = grouped_df.sort_values(by="cross", inplace=False, ascending=False)
    return sorted_df

df_cross_gr_m15 = group_and_sort(df_cross_summary_m15)
df_cross_gr_m30 = group_and_sort(df_cross_summary_m30)
df_cross_gr_h1 = group_and_sort(df_cross_summary_h1)
df_cross_gr_h4 = group_and_sort(df_cross_summary_h4)

In [10]:
df_cross_summary_h4

,pair,num_trades,total_gain,cross
432,EUR_USD,212,-1465,MA_10_MA_20
433,EUR_USD,121,-1582,MA_10_MA_40
434,EUR_USD,113,-622,MA_20_MA_40
435,EUR_USD,129,-1206,MA_30_MA_40
436,EUR_USD,76,190,MA_10_MA_80
...,...,...,...,...
571,GBP_JPY,51,-5071,MA_10_MA_200
572,GBP_JPY,45,-5647,MA_20_MA_200
573,GBP_JPY,41,-6392,MA_30_MA_200
574,GBP_JPY,33,-3451,MA_40_MA_200


In [11]:
def filter_ma_cross(df, cross_value):
    return df[df['cross'] == cross_value]

cross_value = "MA_40_MA_200"
temp = filter_ma_cross(df_ma_res_h4, cross_value)


In [12]:
temp[temp.total_gain > 0]

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
454,EUR_USD,23,614,26,-303,628,MA_200,MA_40,MA_40_MA_200,H4
526,USD_JPY,21,2053,97,-497,1210,MA_200,MA_40,MA_40_MA_200,H4
550,GBP_USD,27,4,0,-446,772,MA_200,MA_40,MA_40_MA_200,H4


In [13]:
temp

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
454,EUR_USD,23,614,26,-303,628,MA_200,MA_40,MA_40_MA_200,H4
478,EUR_GBP,36,-2033,-56,-152,171,MA_200,MA_40,MA_40_MA_200,H4
502,EUR_JPY,27,-1937,-71,-488,1133,MA_200,MA_40,MA_40_MA_200,H4
526,USD_JPY,21,2053,97,-497,1210,MA_200,MA_40,MA_40_MA_200,H4
550,GBP_USD,27,4,0,-446,772,MA_200,MA_40,MA_40_MA_200,H4
574,GBP_JPY,33,-3451,-104,-985,1675,MA_200,MA_40,MA_40_MA_200,H4


In [14]:
total_pairs = len(df_cross_summary_h4)

for c in df_cross_summary_h4['cross'].unique():
    temp = df_cross_summary_h4[df_cross_summary_h4['cross'] == c]
    pg = temp[temp['total_gain'] > 0].shape[0]
    print(f"{c} pg: {pg} percent gain: {(pg / total_pairs) * 100:.2f}%")


MA_10_MA_20 pg: 3 percent gain: 2.08%
MA_10_MA_40 pg: 2 percent gain: 1.39%
MA_20_MA_40 pg: 1 percent gain: 0.69%
MA_30_MA_40 pg: 2 percent gain: 1.39%
MA_10_MA_80 pg: 4 percent gain: 2.78%
MA_20_MA_80 pg: 3 percent gain: 2.08%
MA_30_MA_80 pg: 3 percent gain: 2.08%
MA_40_MA_80 pg: 4 percent gain: 2.78%
MA_50_MA_80 pg: 4 percent gain: 2.78%
MA_10_MA_120 pg: 3 percent gain: 2.08%
MA_20_MA_120 pg: 3 percent gain: 2.08%
MA_30_MA_120 pg: 3 percent gain: 2.08%
MA_40_MA_120 pg: 3 percent gain: 2.08%
MA_50_MA_120 pg: 3 percent gain: 2.08%
MA_10_MA_150 pg: 2 percent gain: 1.39%
MA_20_MA_150 pg: 3 percent gain: 2.08%
MA_30_MA_150 pg: 3 percent gain: 2.08%
MA_40_MA_150 pg: 3 percent gain: 2.08%
MA_50_MA_150 pg: 3 percent gain: 2.08%
MA_10_MA_200 pg: 3 percent gain: 2.08%
MA_20_MA_200 pg: 3 percent gain: 2.08%
MA_30_MA_200 pg: 3 percent gain: 2.08%
MA_40_MA_200 pg: 3 percent gain: 2.08%
MA_50_MA_200 pg: 2 percent gain: 1.39%


In [15]:
cross = "MA_40_MA_200"


df_MA_40_MA_200 = df_ma_res_h4[df_ma_res_h4.cross == cross].copy()

df_MA_40_MA_200.sort_values(by=['total_gain'], ascending=False, inplace=True)


In [16]:
df_MA_40_MA_200

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,ma_l,ma_s,cross,granularity
526,USD_JPY,21,2053,97,-497,1210,MA_200,MA_40,MA_40_MA_200,H4
454,EUR_USD,23,614,26,-303,628,MA_200,MA_40,MA_40_MA_200,H4
550,GBP_USD,27,4,0,-446,772,MA_200,MA_40,MA_40_MA_200,H4
502,EUR_JPY,27,-1937,-71,-488,1133,MA_200,MA_40,MA_40_MA_200,H4
478,EUR_GBP,36,-2033,-56,-152,171,MA_200,MA_40,MA_40_MA_200,H4
574,GBP_JPY,33,-3451,-104,-985,1675,MA_200,MA_40,MA_40_MA_200,H4


In [17]:
df_gains = df_ma_trades_h4[(df_ma_trades_h4.cross == cross) & (df_ma_trades_h4.pair == "USD_JPY")].copy()


In [18]:
df_gains.head(3)

,Unnamed: 0,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MA_40,MA_200,MA_10,MA_80,MA_50,MA_20,MA_150,MA_120,MA_30,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,granularity,pair,GAIN_C,ma_l,ma_s,cross
29665,272,2021-07-15 09:00:00+00:00,14509,109.749,110.090,109.730,110.067,109.742,110.084,109.723,110.060,109.756,110.097,109.736,110.074,110.236775,110.245160,110.1372,110.585375,110.35306,110.25760,110.454040,110.576525,110.176767,-0.008385,0.00746,-1,0.343,-34.3,H4,USD_JPY,-34.3,MA_200,MA_40,MA_40_MA_200
29666,396,2021-08-13 01:00:00+00:00,3634,110.420,110.427,110.371,110.410,110.414,110.420,110.364,110.404,110.427,110.434,110.377,110.417,110.203625,110.189915,110.4122,109.894963,109.97954,110.47765,109.998473,109.950983,110.379433,0.013710,-0.02839,1,-0.878,-87.8,H4,USD_JPY,-122.1,MA_200,MA_40,MA_40_MA_200
29667,410,2021-08-17 09:00:00+00:00,17464,109.306,109.558,109.174,109.532,109.300,109.551,109.164,109.525,109.313,109.565,109.184,109.539,110.105675,110.106015,109.3497,109.822050,110.06066,109.78740,109.930580,109.923617,110.038967,-0.000340,0.00966,-1,0.376,-37.6,H4,USD_JPY,-159.7,MA_200,MA_40,MA_40_MA_200


In [19]:
cp = CandlePlot(df_gains, candles=False)
cp.show_plot(line_traces=['GAIN_C'])

In [20]:
for curr in df_MA_40_MA_200[df_MA_40_MA_200.total_gain > 0].pair.unique():
    temp_gain = df_ma_trades_h4[(df_ma_trades_h4.cross == cross) & (df_ma_trades_h4.pair == curr)].copy()
    print(curr, int(temp_gain.GAIN.sum()))
    cp = CandlePlot(temp_gain, candles=False)
    cp.show_plot(line_traces=['GAIN_C'])

USD_JPY 2053


EUR_USD 614


GBP_USD 4
